In [1]:
from transformers import AutoTokenizer
import torch
from modeling_deepseek_ocr import DeepseekOCRForCausalLM

In [2]:
qwen_path = "/mnt/workspace/models/Qwen/Qwen3-0.6B"
sam_file = "/mnt/workspace/models/facebook/sam-vit-base/sam_vit_b_01ec64_processed.pth"
clip_file = "/mnt/workspace/models/openai/clip-vit-large-patch14/pytorch_model_processed.pth"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(qwen_path)
model = DeepseekOCRForCausalLM.from_pretrained(
    qwen_path,
    torch_dtype="auto",
    device_map="auto"
)

You are using a model of type qwen3 to instantiate a model of type DeepSeekOCR. This is not supported for all configurations of models and can yield errors.
Some weights of DeepseekOCRForCausalLM were not initialized from the model checkpoint at /mnt/workspace/models/Qwen/Qwen3-0.6B and are newly initialized: ['model.image_newline', 'model.projector.bias', 'model.projector.weight', 'model.sam_model.blocks.0.attn.proj.bias', 'model.sam_model.blocks.0.attn.proj.weight', 'model.sam_model.blocks.0.attn.qkv.bias', 'model.sam_model.blocks.0.attn.qkv.weight', 'model.sam_model.blocks.0.attn.rel_pos_h', 'model.sam_model.blocks.0.attn.rel_pos_w', 'model.sam_model.blocks.0.mlp.lin1.bias', 'model.sam_model.blocks.0.mlp.lin1.weight', 'model.sam_model.blocks.0.mlp.lin2.bias', 'model.sam_model.blocks.0.mlp.lin2.weight', 'model.sam_model.blocks.0.norm1.bias', 'model.sam_model.blocks.0.norm1.weight', 'model.sam_model.blocks.0.norm2.bias', 'model.sam_model.blocks.0.norm2.weight', 'model.sam_model.blocks

In [4]:
sam_state_dict = torch.load(sam_file)
model.model.sam_model.load_state_dict(sam_state_dict, strict=False)


_IncompatibleKeys(missing_keys=['net_2.weight', 'net_3.weight'], unexpected_keys=[])

In [5]:
clip_state_dict = torch.load(clip_file)
model.model.vision_model.load_state_dict(clip_state_dict, strict=False)

<All keys matched successfully>

In [6]:
model.model.sam_model.state_dict()['blocks.0.attn.proj.weight']

tensor([[-0.0038,  0.0193,  0.0006,  ...,  0.0039, -0.0237, -0.0004],
        [-0.0127,  0.0071, -0.0154,  ..., -0.0171, -0.0293, -0.0258],
        [ 0.0105, -0.0062, -0.0084,  ..., -0.0066,  0.0157, -0.0396],
        ...,
        [-0.0033, -0.0087,  0.0045,  ..., -0.0479, -0.0259, -0.0090],
        [ 0.0052, -0.0121, -0.0056,  ...,  0.0063,  0.0052,  0.0098],
        [ 0.0094, -0.0138, -0.0194,  ..., -0.0166,  0.0060, -0.0520]],
       dtype=torch.bfloat16)

In [7]:
model.model.vision_model.state_dict()['transformer.layers.0.self_attn.qkv_proj.weight']

tensor([[-7.0572e-05, -1.6499e-04, -7.1049e-05,  ...,  4.5166e-03,
         -2.9175e-02, -7.8201e-05],
        [-1.3733e-04,  1.2159e-04,  4.2439e-05,  ..., -1.6632e-03,
          3.1494e-02,  7.4387e-05],
        [ 4.8065e-04,  7.7820e-04, -1.0967e-04,  ..., -1.6846e-02,
          4.2969e-02,  1.5163e-04],
        ...,
        [ 2.1267e-04,  4.1008e-04, -7.2479e-05,  ...,  4.7913e-03,
         -1.7319e-03, -6.6280e-05],
        [ 3.0518e-04, -4.4346e-05, -2.2697e-04,  ...,  1.1536e-02,
          3.3417e-03,  7.4863e-05],
        [-2.8849e-05,  4.5967e-04,  9.3460e-05,  ..., -1.1292e-02,
          3.7689e-03, -7.7724e-05]], dtype=torch.bfloat16)

In [12]:
prompt = "你是谁"
messages = [
    {"role": "user", "content": prompt},
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True, # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
model_inputs

{'input_ids': tensor([[151644,    872,    198, 105043, 100165, 151645,    198, 151644,  77091,
            198]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [13]:
# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=10
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parse thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)

thinking content: 
content: <think>
嗯，用户问我是谁。我
